Create external database

In [ ]:
IF NOT EXISTS (SELECT * FROM master.dbo.sysdatabases WHERE name = 'externalDB')
    BEGIN
        CREATE DATABASE externalDB
END;

SELECT * FROM master.dbo.sysdatabases WHERE name = 'externalDB';
USE externalDB;
GO

Create master key

In [ ]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Password1234';

Create external data source

In [ ]:
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://controller-svc/default');

Create external table

In [ ]:
BEGIN TRY
    BEGIN TRANSACTION Ta88b9c3f6b794fb7ad2e4bcfa6bb95d
        USE [externalDB];
        CREATE EXTERNAL FILE FORMAT [airline_delay_causes]
            WITH (FORMAT_TYPE = DELIMITEDTEXT, FORMAT_OPTIONS (FIELD_TERMINATOR = N',', STRING_DELIMITER = N'"', FIRST_ROW = 2));
        CREATE EXTERNAL TABLE [dbo].[airline_delay_causes]
        (
            [year] smallint NOT NULL,
            [month] tinyint NOT NULL,
            [carrier] nvarchar(50) NOT NULL,
            [carrier_name] nvarchar(50) NOT NULL,
            [airport] nvarchar(50) NOT NULL,
            [airport_name] nvarchar(100) NOT NULL,
            [arr_flights] float NOT NULL,
            [arr_del15] float NOT NULL,
            [carrier_ct] float NOT NULL,
            [weather_ct] float NOT NULL,
            [nas_ct] float NOT NULL,
            [security_ct] float NOT NULL,
            [late_aircraft_ct] float NOT NULL,
            [arr_cancelled] float NOT NULL,
            [arr_diverted] float NOT NULL,
            [arr_delay] float NOT NULL,
            [carrier_delay] float NOT NULL,
            [weather_delay] float NOT NULL,
            [nas_delay] float NOT NULL,
            [security_delay] float NOT NULL,
            [late_aircraft_delay] float NOT NULL,
            [column22] nvarchar(1)
        )
        WITH (LOCATION = N'/Data/66135976_82020_2429_airline_delay_causes.csv', DATA_SOURCE = [SqlStoragePool], FILE_FORMAT = [airline_delay_causes]);
    COMMIT TRANSACTION Ta88b9c3f6b794fb7ad2e4bcfa6bb95d
END TRY
BEGIN CATCH
    IF @@TRANCOUNT > 0
        ROLLBACK TRANSACTION Ta88b9c3f6b794fb7ad2e4bcfa6bb95d
    DECLARE @ErrorMessage NVARCHAR(max) = ERROR_MESSAGE();
    DECLARE @ErrorSeverity INT = ERROR_SEVERITY();
    DECLARE @ErrorState INT = ERROR_STATE();
    RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState);
END CATCH;


Query data from table

In [ ]:
SELECT TOP (1000) [year]
      ,[month]
      ,[carrier]
      ,[carrier_name]
      ,[airport]
      ,[airport_name]
      ,[arr_flights]
      ,[arr_del15]
      ,[carrier_ct]
      ,[weather_ct]
      ,[nas_ct]
      ,[security_ct]
      ,[late_aircraft_ct]
      ,[arr_cancelled]
      ,[arr_diverted]
      ,[arr_delay]
      ,[carrier_delay]
      ,[weather_delay]
      ,[nas_delay]
      ,[security_delay]
      ,[late_aircraft_delay]
      ,[column22]
  FROM [externalDB].[dbo].[airline_delay_causes]

 Drop the table and database (for cleanup purposes)

In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[importedData].[students]') AND type in (N'U'))
DROP EXTERNAL TABLE [importedData].[students]
GO

USE master;
GO

ALTER DATABASE externalDB SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
GO

DROP DATABASE externalDB;
GO